In [24]:
import os
import numpy as np
import pandas as pd

<h3>Create an DataFramee</h3>

In [25]:
# Load the .npy file
data = np.load('Data_Set.npy',allow_pickle=True)

# Create a dataframe from the numpy array
df = pd.DataFrame(data)

df.columns = ['target_type', 'segment_data', 'range_meters', 'time_seconds', 'dataset_split', 'edge_indicator']

df.head()

,target_type,segment_data,range_meters,time_seconds,dataset_split,edge_indicator
0,[D1],"[[(-0.0803837295206183-0.024996274181930858j),...","[[62.26415142983489], [62.26415142983489], [62...","[[0.04242135490599999], [0.14313034690999998],...","[[2], [3], [1], [3], [1], [1], [1], [1], [3], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,[D1],"[[(0.1592469814114204-0.3764412575384074j), (0...","[[116.03773675560139], [116.03773675560139], [...","[[0.04531640558899999], [0.148743200275], [0.2...","[[3], [1], [3], [1], [1], [1], [2], [2], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
2,[D1],"[[(-0.36744600235662883-0.17434743507810269j),...","[[75.47169870283017], [75.47169870283017], [75...","[[0.04389842158099999], [0.14448924825099999],...","[[1], [1], [1], [1], [2], [1], [1], [3], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
3,[D1],"[[(-0.7918969406816826-0.0388248167751634j), (...","[[145.28302000294806], [145.28302000294806], [...","[[0.04372117357999999], [0.14372117358], [0.24...","[[1], [2], [3], [1], [3], [1], [3], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
4,[D1],"[[(0.08721619353018038-0.05631563733996364j), ...","[[22.641509610849056], [22.641509610849056], [...","[[0.03592226153599999], [0.13604042687], [0.23...","[[1], [1], [3], [1], [3], [1], [1], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."


<h3>Remove the unwanted columns</h3>

In [26]:
# Keep only target_type and segment_data columns
df = df[['target_type', 'segment_data']]

<h3>Label encoding the target_types</h3>

In [27]:
# Define valid target types
target_mapping = {
    'D1': 1,
    'D2': 1,
    'D3': 1,
    'D4': 1,
    'D5': 1,
    'D6': 1,
    'seagull': 0,
    'pigeon': 0,
    'raven': 0,
    'black-headed gull': 0,
    'seagull and black-headed gull': 0,
    'heron': 0
}

# Keep only rows where target_type is in our mapping
df = df[df['target_type'].apply(lambda x: x[0] in target_mapping)]

# Now map the values to binary
df['target_type'] = df['target_type'].apply(lambda x: target_mapping[x[0]])


In [28]:
# Get value counts and percentages
target_summary = df['target_type'].value_counts()
target_percentages = df['target_type'].value_counts(normalize=True) * 100

print("Target Type Distribution:")
print("\nCounts:")
print(target_summary)


Target Type Distribution:

Counts:
target_type
0    56
1    44
Name: count, dtype: int64


In [29]:
df.head()

,target_type,segment_data
0,1,"[[(-0.0803837295206183-0.024996274181930858j),..."
1,1,"[[(0.1592469814114204-0.3764412575384074j), (0..."
2,1,"[[(-0.36744600235662883-0.17434743507810269j),..."
3,1,"[[(-0.7918969406816826-0.0388248167751634j), (..."
4,1,"[[(0.08721619353018038-0.05631563733996364j), ..."


<h3>Save each segment data in as a separate file</h3>

In [30]:
# Create directories if they don't exist
os.makedirs('Drone_segment_only_data', exist_ok=True)
os.makedirs('Bird_segment_only_data', exist_ok=True)

# Counter for file naming
drone_counter = 1
bird_counter = 1

# Iterate through the dataframe
for index, row in df.iterrows():
    if row['target_type'] == 1:
        # Save drone data
        np.save(f'Drone_segment_only_data/drone{drone_counter}.npy', row['segment_data'])
        drone_counter += 1
    else:
        # Save bird data
        np.save(f'Bird_segment_only_data/bird{bird_counter}.npy', row['segment_data'])
        bird_counter += 1

print(f"Saved {drone_counter-1} drone files and {bird_counter-1} bird files")


Saved 44 drone files and 56 bird files


In [21]:
''' To check the dimension of the data being saved in .npy format '''

# Load a numpy file
data = np.load(r'D:\Project\SIH\Classification_Backend\Micro_Doppler_Based_Target_Classification\Bird_segment_only_data\bird13.npy')

# Convert the numpy array to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output_file.csv', index=False)

# Print the shape of the numpy array and the DataFrame
print(f"Numpy array shape: {data.shape}")
print(f"DataFrame shape: {df.shape}")

Numpy array shape: (1280, 32)
DataFrame shape: (1280, 32)


<h3>Store each segment as a separate npy file </h3>

In [32]:
def process_segments(input_dir, output_base_dir, file_prefix):
    os.makedirs(output_base_dir, exist_ok=True)
    
    # Get list of all files
    files = os.listdir(input_dir)
    
    # Process each file
    for file in files:
        if file.endswith('.npy'):
            # Get file name without extension
            number = file.split('.')[0]  # Get bird1/drone1 etc
            
            # Create specific folder for this file's segments
            file_output_dir = f'{output_base_dir}/{number}'
            os.makedirs(file_output_dir, exist_ok=True)
            
            # Load the segment data
            data = np.load(f'{input_dir}/{file}')
            
            # Split and save each column
            for col in range(data.shape[1]):
                column_data = data[:, col].reshape(1, -1)
                np.save(f'{file_output_dir}/{number}_{col+1}seg.npy', column_data)
    
    print(f"Successfully split {len(files)} {file_prefix} files into individual segments")

# Process bird segments
process_segments('Bird_segment_only_data', 'Bird_single_segment_data', 'bird')

# Process drone segments
process_segments('Drone_segment_only_data', 'Drone_single_segment_data', 'drone')


Successfully split 56 bird files into individual segments
Successfully split 44 drone files into individual segments


<h3>Read the number of files in the Drones and Birds</h3>

In [33]:
def count_segments(base_dir):
    total_count = 0
    # Iterate through each subdirectory
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            # Count .npy files in each subdirectory
            count = len([f for f in os.listdir(folder_path) if f.endswith('.npy')])
            total_count += count
    return total_count

# Count files in drone and bird segments folders
drone_files = count_segments('Drone_single_segment_data')
bird_files = count_segments('Bird_single_segment_data')

print(f"Number of drone segment files: {drone_files}")
print(f"Number of bird segment files: {bird_files}")
print(f"Total number of segment files: {drone_files + bird_files}")


Number of drone segment files: 58768
Number of bird segment files: 7792
Total number of segment files: 66560


In [34]:
# Load a numpy file
data = np.load(r'D:\Project\SIH\Classification_Backend\Micro_Doppler_Based_Target_Classification\Data_Sets\bird56_segment_2.npy')

# Convert the numpy array to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output_file.csv', index=False)

# Print the shape of the numpy array and the DataFrame
print(f"Numpy array shape: {data.shape}")
print(f"DataFrame shape: {df.shape}")

Numpy array shape: (1, 1280)
DataFrame shape: (1, 1280)


In [35]:
# Load a numpy file
data = np.load(r'D:\Project\SIH\Classification_Backend\Micro_Doppler_Based_Target_Classification\Drone_single_segment_data\drone37\drone37_21seg.npy')

# Convert the numpy array to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output_file.csv', index=False)

# Print the shape of the numpy array and the DataFrame
print(f"Numpy array shape: {data.shape}")
print(f"DataFrame shape: {df.shape}")

Numpy array shape: (1, 1280)
DataFrame shape: (1, 1280)
